# DETR Smoke Detection Training on Pyronear Dataset (Colab Ready)

This notebook trains a [facebook/detr-resnet-50](https://huggingface.co/facebook/detr-resnet-50) model on the Pyronear smoke dataset, downloading data directly from Hugging Face. It uses PyTorch Lightning and torchmetrics to provide detailed YOLO-style metrics (mAP@0.5, mAP@0.5:0.95, precision, recall, etc.) during training.


## 📦 Setup and Install Dependencies

In [ ]:
!pip install -q torch torchvision pytorch-lightning torchmetrics transformers datasets huggingface_hub pycocotools opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 35.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 92.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.1/823.1 kB 53.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 961.5/961.5 kB 55.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## 📚 Imports

In [ ]:
import os
import json
from PIL import Image
import torch
from torchvision.datasets import CocoDetection
from torchvision import transforms
from torch.utils.data import DataLoader
from transformers import DetrImageProcessor, DetrForObjectDetection
import matplotlib.pyplot as plt
from datasets import load_dataset
import pytorch_lightning as pl
from torchmetrics.detection.mean_ap import MeanAveragePrecision


## 📥 Download Pyronear Dataset from Hugging Face
We use the [datasets](https://huggingface.co/docs/datasets) library to download the Pyronear smoke dataset.

In [ ]:
# Change to the correct dataset repo if needed

dataset = load_dataset('pyronear/pyro-sdis', split='train')
# If the dataset is not in COCO format, conversion logic will be added below.

README.md:   0%|          | 0.00/7.37k [00:00<?, ?B/s]

train-00000-of-00006.parquet:   0%|          | 0.00/481M [00:00<?, ?B/s]

train-00001-of-00006.parquet:   0%|          | 0.00/485M [00:00<?, ?B/s]

train-00002-of-00006.parquet:   0%|          | 0.00/482M [00:00<?, ?B/s]

train-00003-of-00006.parquet:   0%|          | 0.00/483M [00:00<?, ?B/s]

train-00004-of-00006.parquet:   0%|          | 0.00/480M [00:00<?, ?B/s]

train-00005-of-00006.parquet:   0%|          | 0.00/483M [00:00<?, ?B/s]

val-00000-of-00001.parquet:   0%|          | 0.00/390M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/29537 [00:00<?, ? examples/s]

Generating val split:   0%|          | 0/4099 [00:00<?, ? examples/s]

In [ ]:
val_dataset = load_dataset('pyronear/pyro-sdis', split='val')

In [4]:
from google.colab import drive
import os
drive.mount('/content/drive')
CHECKPOINT_DIR = '/content/drive/MyDrive/detr_checkpoints'
os.makedirs(CHECKPOINT_DIR, exist_ok=True)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 🔁 Convert YOLO to COCO format (if needed)
If the dataset is in YOLO format, convert it to COCO format for DETR compatibility.

In [ ]:
 import json
from tqdm import tqdm

def convert_yolo_to_coco(dataset, class_list):
    coco_dict = {
        "images": [],
        "annotations": [],
        "categories": []
    }
    annotation_id = 1
    for idx, data in enumerate(tqdm(dataset)):
        image_id = idx + 1
        img = data['image']
        width, height = img.size
        coco_dict["images"].append({
            "id": image_id,
            "file_name": data['image_name'],
            "width": width,
            "height": height
        })
        # Handle multiple annotations per image if needed
        annos = data['annotations'].strip().split('\n')
        for anno in annos:
            parts = anno.strip().split()
            if len(parts) != 5:
                continue  # skip malformed lines
            class_id, x_center, y_center, w, h = map(float, parts)
            # Convert YOLO to COCO bbox
            x = (x_center - w/2) * width
            y = (y_center - h/2) * height
            w_box = w * width
            h_box = h * height
            coco_dict["annotations"].append({
                "id": annotation_id,
                "image_id": image_id,
                "category_id": int(class_id) + 1,  # COCO ids start at 1
                "bbox": [x, y, w_box, h_box],
                "area": w_box * h_box,
                "iscrowd": 0
            })
            annotation_id += 1
    # Add categories
    for i, name in enumerate(class_list):
        coco_dict["categories"].append({
            "id": i + 1,
            "name": name
        })
    return coco_dict

# Example usage:
class_list = ["smoke"]  # Update if you have more classes
coco_dict = convert_yolo_to_coco(dataset, class_list)


# Save to file if needed:
with open("annotations.json", "w") as f:
    json.dump(coco_dict, f)

100%|██████████| 29537/29537 [02:04<00:00, 237.48it/s]


In [ ]:
coco_dict_val = convert_yolo_to_coco(val_dataset, class_list=class_list)
# Save to file if needed:
with open("annotations.json", "w") as f:
    json.dump(coco_dict_val, f)

100%|██████████| 4099/4099 [00:16<00:00, 250.94it/s]


## 🗂️ Prepare DataLoaders
Wrap the COCO dataset for use with PyTorch Lightning.

In [ ]:
from torch.utils.data import Dataset, DataLoader

class InMemoryCocoDataset(Dataset):
    def __init__(self, dataset, coco_dict, transform=None):
        self.dataset = dataset
        self.coco_dict = coco_dict
        self.transform = transform
        # Map image_id to annotations
        self.ann_map = {}
        for ann in coco_dict['annotations']:
            self.ann_map.setdefault(ann['image_id'], []).append(ann)
        # List of images
        self.images = coco_dict['images']

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_info = self.images[idx]
        image_id = img_info['id']
        img = self.dataset[idx]['image']
        anns = self.ann_map.get(image_id, [])
        if len(anns) > 0:
            boxes = torch.tensor([ann['bbox'] for ann in anns], dtype=torch.float32)
            labels = torch.tensor([ann['category_id'] for ann in anns], dtype=torch.int64)
        else:
            boxes = torch.zeros((0, 4), dtype=torch.float32)
            labels = torch.zeros((0,), dtype=torch.int64)
        target = {
            'boxes': boxes,
            'labels': labels
        }
        if self.transform:
            img = self.transform(img)
        return img, target

# Usage:
transform = transforms.Compose([transforms.ToTensor()])
custom_dataset = InMemoryCocoDataset(dataset, coco_dict, transform=transform)
dataloader = DataLoader(custom_dataset, batch_size=4, shuffle=True, num_workers=2, collate_fn=lambda x: tuple(zip(*x)))

custom_dataset_val = InMemoryCocoDataset(val_dataset, coco_dict_val, transform=transform)
val_dataloader = DataLoader(custom_dataset_val, batch_size=4, shuffle=True, num_workers=2, collate_fn=lambda x: tuple(zip(*x)))

## ⚡ PyTorch Lightning Module with YOLO-style Metrics
We use torchmetrics' MeanAveragePrecision for mAP@0.5, mAP@0.5:0.95, precision, recall, etc.

In [ ]:
from transformers import DetrForObjectDetection, DetrConfig, DetrImageProcessor
import torch
import pytorch_lightning as pl
from torchmetrics.detection.mean_ap import MeanAveragePrecision

from pytorch_lightning.callbacks import ModelCheckpoint

checkpoint_callback = ModelCheckpoint(
    dirpath=CHECKPOINT_DIR,                   # Save into your Google Drive
    filename='detr-{epoch:02d}-{val_map_epoch:.2f}',  # filename format
    monitor='val_map_epoch',                   # OR 'val_loss' if you prefer
    mode='max',                                # 'max' for mAP, 'min' for loss
    save_top_k=1,                              # Only keep the best checkpoint
    verbose=True
)


class DETRLightningModule(pl.LightningModule):
    def __init__(self, num_classes):
        super().__init__()
        # Load config and set num_labels
        config = DetrConfig.from_pretrained('facebook/detr-resnet-50')
        config.num_labels = num_classes
        self.model = DetrForObjectDetection.from_pretrained(
            'facebook/detr-resnet-50', config=config, ignore_mismatched_sizes=True
        )
        self.processor = DetrImageProcessor.from_pretrained('facebook/detr-resnet-50')
        self.map_metric = MeanAveragePrecision(class_metrics=True)

    def forward(self, pixel_values):
        return self.model(pixel_values)

    def training_step(self, batch, batch_idx):
        images, targets = batch
        encoding = self.processor(images, return_tensors="pt").to(self.device)
        labels = [{"class_labels": t['labels'], "boxes": t['boxes']} for t in targets]
        outputs = self.model(**encoding, labels=labels)
        loss = outputs.loss

        # Handle target sizes for PIL or tensor images
        if isinstance(images[0], torch.Tensor):
            target_sizes = torch.stack([torch.tensor(img.shape[-2:]) for img in images]).to(self.device)
        else:
            target_sizes = torch.tensor([img.size[::-1] for img in images]).to(self.device)

        results = self.processor.post_process_object_detection(
            outputs, target_sizes=target_sizes, threshold=0.5
        )

        # Move predictions to device
        for r in results:
            r["boxes"] = r["boxes"].to(self.device)
            r["labels"] = r["labels"].to(self.device)
            r["scores"] = r["scores"].to(self.device)

        # Move targets to device
        formatted_targets = []
        for t in targets:
            formatted_targets.append({
                "boxes": t["boxes"].to(self.device),
                "labels": t["labels"].to(self.device)
            })

        self.map_metric.update(results, formatted_targets)
        metrics = self.map_metric.compute()
        if 'map_per_class' in metrics and isinstance(metrics['map_per_class'], torch.Tensor) and metrics['map_per_class'].numel() > 1:
            metrics['map_per_class_mean'] = metrics['map_per_class'][metrics['map_per_class'] != -1].mean().item()
            del metrics['map_per_class']
        scalar_metrics = {k: (v.item() if isinstance(v, torch.Tensor) and v.numel() == 1 else v) for k, v in metrics.items() if isinstance(v, (float, int)) or (isinstance(v, torch.Tensor) and v.numel() == 1)}
        self.log_dict(scalar_metrics, prog_bar=True, on_step=True, on_epoch=True)
        return loss

    def validation_step(self, batch, batch_idx):
        images, targets = batch
        encoding = self.processor(images, return_tensors="pt").to(self.device)
        labels = [{"class_labels": t['labels'], "boxes": t['boxes']} for t in targets]
        outputs = self.model(**encoding, labels=labels)
        loss = outputs.loss

        # Handle target sizes for PIL or tensor images
        if isinstance(images[0], torch.Tensor):
            target_sizes = torch.stack([torch.tensor(img.shape[-2:]) for img in images]).to(self.device)
        else:
            target_sizes = torch.tensor([img.size[::-1] for img in images]).to(self.device)

        results = self.processor.post_process_object_detection(
            outputs, target_sizes=target_sizes, threshold=0.5
        )

        # Move predictions to device
        for r in results:
            r["boxes"] = r["boxes"].to(self.device)
            r["labels"] = r["labels"].to(self.device)
            r["scores"] = r["scores"].to(self.device)

        # Move targets to device
        formatted_targets = []
        for t in targets:
            formatted_targets.append({
                "boxes": t["boxes"].to(self.device),
                "labels": t["labels"].to(self.device)
            })

        self.map_metric.update(results, formatted_targets)
        metrics = self.map_metric.compute()
        if 'map_per_class' in metrics and isinstance(metrics['map_per_class'], torch.Tensor) and metrics['map_per_class'].numel() > 1:
            metrics['map_per_class_mean'] = metrics['map_per_class'][metrics['map_per_class'] != -1].mean().item()
            del metrics['map_per_class']
        scalar_metrics = {k: (v.item() if isinstance(v, torch.Tensor) and v.numel() == 1 else v) for k, v in metrics.items() if isinstance(v, (float, int)) or (isinstance(v, torch.Tensor) and v.numel() == 1)}
        self.log_dict(scalar_metrics, prog_bar=True, on_step=True, on_epoch=True)
        return loss

    def configure_optimizers(self):
        return torch.optim.AdamW(self.parameters(), lr=1e-4)

    def on_train_epoch_end(self):
      train_metrics = self.map_metric.compute()
      self.log_dict({f'train_{k}': v for k, v in train_metrics.items()}, prog_bar=True)
      self.map_metric.reset()

    def on_validation_epoch_end(self):
      val_metrics = self.map_metric.compute()
      self.log_dict({f'val_{k}': v for k, v in val_metrics.items()}, prog_bar=True)
      self.map_metric.reset()




## 🚂 Train the Model
Set up the PyTorch Lightning Trainer and start training.

In [ ]:
print("Train split length:", len(dataset))
print("Train COCO images:", len(coco_dict['images']))
print("Val split length:", len(val_dataset))
print("Val COCO images:", len(coco_dict_val['images']))

Train split length: 29537
Train COCO images: 29537
Val split length: 4099
Val COCO images: 4099


In [ ]:
# Example usage (replace with your actual dataloader and num_classes)
model = DETRLightningModule(num_classes=2)
trainer = pl.Trainer(max_epochs=10, accelerator='gpu' if torch.cuda.is_available() else 'cpu',callbacks=[checkpoint_callback])
trainer.fit(model,dataloader, val_dataloader)


config.json:   0%|          | 0.00/4.59k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/167M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/102M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:2397: UserWarning: for conv1.weight: copying from a non-meta parameter in the checkpoint to a meta parameter in the current model, which is a no-op. (Did you mean to pass `assign=True` to assign items in the state dictionary to their corresponding key in the module instead of copying them in place?)
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:2397: UserWarning: for bn1.weight: copying from a non-meta parameter in the checkpoint to a meta parameter in the current model, which is a no-op. (Did you mean to pass `assign=True` to assign items in the state dictionary to their corresponding key in the module instead of copying them in place?)
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:2397: UserWarning: for bn1.bias: copying from a non-meta parameter in the checkpoint to a meta parameter in the current model, which is a no-op. (Did you mean to pas

preprocessor_config.json:   0%|          | 0.00/290 [00:00<?, ?B/s]

INFO:pytorch_lightning.utilities.rank_zero:Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name       | Type                   | Params | Mode 
--------------------------------------------------------------
0 | model      | DetrForObjectDetection | 41.5 M | eval 
1 | map_metric | MeanAveragePrecision   | 0      | train
--------------------------------------------------------------
41.3 M    Trainable params
222 K     Non-trainable params
41.5 M    Total params
166.008   Total estimated model params size (

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:476: Your `val_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.
It looks like you are trying to rescale already rescaled images. If the input images have pixel values between 0 and 1, set `do_rescale=False` to avoid rescaling them again.
/usr/local/lib/python3.11/dist-packages/pytorch_lightning/utilities/data.py:79: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 3. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
/usr/local/lib/python3.11/dist-packages/pytorch_lightning/trainer/connectors/logger_connector/result.py:213: You called `self.log('val_classes', ...)` in your `on_validation_epoch_end` but the value needs to be floating to be reduced. Converting it to torch.float32. You can silence this warning by converting the value to floating point yours

Training: |          | 0/? [00:00<?, ?it/s]

## 📊 Visualize Metrics and Predictions
Plot or print the metrics after each epoch, and visualize sample predictions.

In [ ]:
# 📊 Plot mAP, Precision, and Recall per epoch from Lightning logs

import matplotlib.pyplot as plt

def plot_metrics(trainer):
    # Extract metrics from the trainer's logger
    metrics = trainer.callback_metrics
    epochs = range(1, trainer.current_epoch + 2)

    # These keys may differ depending on your metric names/logs
    mAP_50 = [metrics.get(f"map_50_epoch_{e}", None) for e in range(trainer.current_epoch + 1)]
    mAP_95 = [metrics.get(f"map_epoch_{e}", None) for e in range(trainer.current_epoch + 1)]
    precision = [metrics.get(f"precision_epoch_{e}", None) for e in range(trainer.current_epoch + 1)]
    recall = [metrics.get(f"recall_epoch_{e}", None) for e in range(trainer.current_epoch + 1)]

    plt.figure(figsize=(10, 6))
    plt.plot(epochs, mAP_50, label="mAP@0.5")
    plt.plot(epochs, mAP_95, label="mAP@0.5:0.95")
    plt.plot(epochs, precision, label="Precision")
    plt.plot(epochs, recall, label="Recall")
    plt.xlabel("Epoch")
    plt.ylabel("Metric Value")
    plt.title("Detection Metrics per Epoch")
    plt.legend()
    plt.grid()
    plt.show()

# Usage example (run after training):
# plot_metrics(trainer)


In [ ]:
# 🖼️ Visualize sample predictions from the trained model

import torch
import matplotlib.pyplot as plt
import numpy as np

def plot_predictions(model, processor, dataloader, device, class_names, num_images=4, score_threshold=0.5):
    model.eval()
    images_shown = 0

    for images, targets in dataloader:
        # Move images to device
        pixel_values = processor(images, return_tensors="pt", padding=True).pixel_values.to(device)
        with torch.no_grad():
            outputs = model(pixel_values=pixel_values)

        # Post-process outputs
        results = processor.post_process_object_detection(outputs, target_sizes=[img.size[::-1] for img in images], threshold=score_threshold)

        for idx, (image, result) in enumerate(zip(images, results)):
            plt.figure(figsize=(8, 6))
            plt.imshow(image)
            ax = plt.gca()
            boxes = result["boxes"].cpu().numpy()
            scores = result["scores"].cpu().numpy()
            labels = result["labels"].cpu().numpy()

            for box, score, label in zip(boxes, scores, labels):
                xmin, ymin, xmax, ymax = box
                ax.add_patch(plt.Rectangle((xmin, ymin), xmax - xmin, ymax - ymin, fill=False, color='red', linewidth=2))
                ax.text(xmin, ymin, f'{class_names[label]}: {score:.2f}', bbox=dict(facecolor='yellow', alpha=0.5), fontsize=10, color='black')
            plt.axis('off')
            plt.show()
            images_shown += 1
            if images_shown >= num_images:
                return

# Usage example (after training):
# class_names = ["background", "smoke"]  # adjust as needed
# plot_predictions(model.model, model.processor, val_dataloader, device="cuda" if torch.cuda.is_available() else "cpu", class_names=class_names)